In [21]:
import pandas as pd
import os

# List of filenames
csv_files = [
    '/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/client1 _bbr_full.csv',
    '/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/client1_cubic_full.csv',
    '/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/client1_pcc.csv',
    '/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/client2_bbr_switch_cubic.csv',
    '/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/client2_pcc.csv',
    '/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/client2_pcc_switch_bbr.csv'
]

# Function to extract CCA from the filename
def get_cca_from_filename(filename, switch_time=40):
    if 'switch' in filename:
        if 'bbr_switch_cubic' in filename:
            return 'BBR', 'CUBIC', switch_time
        elif 'pcc_switch_bbr' in filename:
            return 'PCC', 'BBR', switch_time
    elif 'bbr' in filename:
        return 'BBR', None, None
    elif 'cubic' in filename:
        return 'CUBIC', None, None
    elif 'pcc' in filename:
        return 'PCC', None, None
    else:
        return 'UNKNOWN', None, None

# Initialize an empty list to hold DataFrames
dataframes = []

# Loop through the list of CSV files and read each one into a DataFrame
for file in csv_files:
    if os.path.exists(file):
        df = pd.read_csv(file)
        if 'Timestamp' not in df.columns:
            print(f"Warning: 'Timestamp' column not found in {file}. Skipping this file.")
            continue
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # Ensure Timestamp is in datetime format
        df['time'] = (df['Timestamp'] - df['Timestamp'].min()).dt.total_seconds()  # Create a 'time' column in seconds
        cca1, cca2, switch_time = get_cca_from_filename(file)
        if cca2 is None:
            df['CCA'] = cca1
        else:
            df['CCA'] = [cca1 if t <= switch_time else cca2 for t in df['time']]
        dataframes.append(df)
    else:
        print(f"File not found: {file}")

# Concatenate all DataFrames into a single DataFrame if dataframes list is not empty
if dataframes:
    combined_data = pd.concat(dataframes, ignore_index=True)
    # Save the combined DataFrame to a CSV file
    combined_data.to_csv('/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/combined_data.csv', index=False)
    print("Combined data saved to '/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/combined_data.csv'")
else:
    print("No valid files were found. Please check the file paths and content.")


File not found: /home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/client1 _bbr_full.csv
Combined data saved to '/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/combined_data.csv'


In [22]:
# Fill missing values for numerical columns with mean
for col in combined_data.select_dtypes(include=['float64', 'int64']).columns:
    combined_data[col].fillna(combined_data[col].mean(), inplace=True)

# Fill missing values for categorical columns with mode
for col in combined_data.select_dtypes(include=['object']).columns:
    combined_data[col].fillna(combined_data[col].mode()[0], inplace=True)

# Verify if all missing values are handled
missing_values_post = combined_data.isnull().sum()
print("Missing values after handling:")
print(missing_values_post)


Missing values after handling:
Timestamp      0
Throughput     0
LossRate       0
Latency        0
SendingRate    0
time           0
CCA            0
dtype: int64


/tmp/ipykernel_369022/299680828.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_data[col].fillna(combined_data[col].mean(), inplace=True)
/tmp/ipykernel_369022/299680828.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

In [23]:
from sklearn.preprocessing import StandardScaler

# Select numerical columns for standardization
numerical_cols = combined_data.select_dtypes(include=['float64', 'int64']).columns

# Standardize numerical columns
scaler = StandardScaler()
combined_data[numerical_cols] = scaler.fit_transform(combined_data[numerical_cols])

# Display the first few rows after standardization
print(combined_data.head())


            Timestamp  Throughput   LossRate       Latency  SendingRate  \
0 2024-05-30 07:31:40   -2.063979  16.292751  8.473640e-17    -3.328244   
1 2024-05-30 07:31:41   -0.170823   0.580359  8.473640e-17    -0.282757   
2 2024-05-30 07:31:42    0.504422  -0.011853  8.473640e-17    -0.916587   
3 2024-05-30 07:31:43    1.590342  -0.047357  8.473640e-17    -0.755940   
4 2024-05-30 07:31:44    1.758314  -0.060487  8.473640e-17    -0.867777   

       time    CCA  
0 -1.712318  CUBIC  
1 -1.680774  CUBIC  
2 -1.649230  CUBIC  
3 -1.617685  CUBIC  
4 -1.586141  CUBIC  


In [24]:
from sklearn.preprocessing import StandardScaler

# Select numerical columns for standardization
numerical_cols = combined_data.select_dtypes(include=['float64', 'int64']).columns

# Standardize numerical columns
scaler = StandardScaler()
combined_data[numerical_cols] = scaler.fit_transform(combined_data[numerical_cols])

# Display the first few rows after standardization
print(combined_data.head())


            Timestamp  Throughput   LossRate       Latency  SendingRate  \
0 2024-05-30 07:31:40   -2.063979  16.292751  3.151223e-17    -3.328244   
1 2024-05-30 07:31:41   -0.170823   0.580359  3.151223e-17    -0.282757   
2 2024-05-30 07:31:42    0.504422  -0.011853  3.151223e-17    -0.916587   
3 2024-05-30 07:31:43    1.590342  -0.047357  3.151223e-17    -0.755940   
4 2024-05-30 07:31:44    1.758314  -0.060487  3.151223e-17    -0.867777   

       time    CCA  
0 -1.712318  CUBIC  
1 -1.680774  CUBIC  
2 -1.649230  CUBIC  
3 -1.617685  CUBIC  
4 -1.586141  CUBIC  


In [25]:
# Split the data into training and testing sets
split_index = int(0.8 * len(combined_data))
train_data = combined_data.iloc[:split_index]
test_data = combined_data.iloc[split_index:]

# Save training data to CSV
train_data.to_csv('/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/train_data.csv', index=False)

# Save testing data to CSV
test_data.to_csv('/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/test_data.csv', index=False)

In [26]:
import gym
from gym import spaces
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import random
import matplotlib.pyplot as plt

class CongestionControlEnv(gym.Env):
    def __init__(self, data, thresholds):
        super(CongestionControlEnv, self).__init__()
        self.data = data
        self.thresholds = thresholds
        self.observation_space = spaces.Box(low=0, high=1, shape=(4,), dtype=np.float32)
        self.action_space = spaces.Discrete(2)
        self.current_step = 0
        self.state = None
        self.current_cca = 'default'
        self.available_ccas = ['cca1', 'cca2', 'cca3']



    def reset(self):
        self.current_step = 0
        self.state = self.data.iloc[self.current_step][['Throughput', 'Latency', 'LossRate', 'SendingRate']].values
        self.current_cca = 'default'
        return self.state

    def step(self, action):
        if action == 1:
            best_cca = self.evaluate_best_cca()
            if best_cca != self.current_cca:
                self.current_cca = best_cca
                print(f"Switched to congestion control algorithm: {self.current_cca}")

        self.current_step += 1
        done = self.current_step >= len(self.data)
        if not done:
            self.state = self.data.iloc[self.current_step][['Throughput', 'Latency', 'LossRate', 'SendingRate']].values
        else:
            self.state = None

        reward = self._calculate_reward(action)
        info = {'action': action, 'cca': self.current_cca}

        return self.state, reward, done, info

    def evaluate_best_cca(self):
        current_state = self.state
        historical_data = self.get_historical_data()

        if historical_data is not None and len(historical_data) > 0:
            combined_state = 0.8 * current_state + 0.2 * np.mean(historical_data, axis=0)
        else:
            combined_state = current_state

        best_cca = self.choose_best_cca_from_evaluation(combined_state)
        return best_cca

    def choose_best_cca_from_evaluation(self, combined_state):
        weights = {'Throughput': 0.7, 'Latency': 0.2, 'LossRate': 0.1}
        cca_scores = {cca: sum(weights[metric] * combined_state[i] for i, metric in enumerate(weights)) for cca in self.available_ccas}
        best_cca = max(cca_scores, key=cca_scores.get)
        return best_cca


    def _calculate_reward(self, action):
        if self.state is None:  # Handle case where state is None
            return 0  # Return zero reward if state is None
        disparities = np.abs(self.state - self.data.iloc[self.current_step-1][['Throughput', 'Latency', 'LossRate', 'SendingRate']].values)
        reward = 0
        if np.any(disparities > self.thresholds):
            reward += np.log10(0.059)  # High disparities reward
        else:
            reward -= np.log10(0.01)  # Low disparities penalty
        if action == 1:
            reward += np.log10(0.05) # Switching penalty
        return reward


    def get_historical_data(self):
        return self.data.iloc[:self.current_step][['Throughput', 'Latency', 'LossRate', 'SendingRate']].values

class DQNAgent:
    def __init__(self, state_shape, action_space):
        self.state_shape = state_shape
        self.action_space = action_space
        self.model = self.build_model()
        self.replay_buffer = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.loss_history = []
        self.accuracy_history = []
        self.save_path = '/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/my_dqn_model2.h5'  # Specify your desired save path here

    def build_model(self):
        model = tf.keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=self.state_shape),
            layers.Dense(64, activation='relu'),
            layers.Dense(self.action_space, activation='linear')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['accuracy'])
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_space)
        state_tensor = tf.convert_to_tensor(state, dtype=tf.float32)
        act_values = self.model.predict(state_tensor)
        return np.argmax(act_values[0])

    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self, batch_size):
        if len(self.replay_buffer) < batch_size:
            return
        minibatch = random.sample(self.replay_buffer, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                next_state_tensor = tf.convert_to_tensor(next_state, dtype=tf.float32)
                target = reward + self.gamma * np.amax(self.model.predict(next_state_tensor)[0])
            state_tensor = tf.convert_to_tensor(state, dtype=tf.float32)
            target_f = self.model.predict(state_tensor)
            target_f[0][action] = target
            history = self.model.fit(state_tensor, target_f, epochs=1, verbose=0)
            self.loss_history.append(history.history['loss'][0])
            self.accuracy_history.append(history.history['accuracy'][0])
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save_model(self):
        self.model.save(self.save_path)

    def load_model(self, file_path):
        self.model = tf.keras.models.load_model(file_path)


In [27]:
# Training loop
EPISODES =1000
BATCH_SIZE = 32

# Example dataset, replace with actual data
combined_data = np.random.rand(100, 4)
combined_data = pd.DataFrame(combined_data, columns=['Throughput', 'Latency', 'LossRate', 'SendingRate'])

env = CongestionControlEnv(data=combined_data, thresholds=[0.1, 0.1, 0.1, 0.1])
state_shape = (env.observation_space.shape[0],)
action_space = env.action_space.n
agent = DQNAgent(state_shape, action_space)

# Initialize CSV file for logging
with open('training_log.csv', 'w', newline='') as csvfile:
    fieldnames = ['episode', 'total_reward', 'loss', 'accuracy']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

rewards = []
losses = []
accuracies = []

# Plotting setup
plt.ion()
fig, axs = plt.subplots(3, 1, figsize=(10, 15))
reward_line, = axs[0].plot([], [], label='Reward')
loss_line, = axs[1].plot([], [], label='Loss')
accuracy_line, = axs[2].plot([], [], label='Accuracy')

axs[0].set_xlabel('Episode')
axs[0].set_ylabel('Total Reward')
axs[0].set_title('Training Rewards')
axs[1].set_xlabel('Episode')
axs[1].set_ylabel('Loss')
axs[1].set_title('Training Loss')
axs[2].set_xlabel('Episode')
axs[2].set_ylabel('Accuracy')
axs[2].set_title('Training Accuracy')

def update_plot():
    reward_line.set_xdata(range(len(rewards)))
    reward_line.set_ydata(rewards)
    axs[0].relim()
    axs[0].autoscale_view()

    loss_line.set_xdata(range(len(losses)))
    loss_line.set_ydata(losses)
    axs[1].relim()
    axs[1].autoscale_view()

    accuracy_line.set_xdata(range(len(accuracies)))
    accuracy_line.set_ydata(accuracies)
    axs[2].relim()
    axs[2].autoscale_view()

    plt.draw()
    plt.pause(0.01)

for episode in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_shape[0]])
    total_reward = 0
    done = False
    while not done:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        if next_state is not None:
            next_state = np.reshape(next_state, [1, state_shape[0]])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
        total_reward += reward
    agent.train(BATCH_SIZE)
    rewards.append(total_reward)
    losses.append(np.mean(agent.loss_history[-BATCH_SIZE:]))
    accuracies.append(np.mean(agent.accuracy_history[-BATCH_SIZE:]))

    with open('training_log.csv', 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'episode': episode + 1, 'total_reward': total_reward, 'loss': losses[-1], 'accuracy': accuracies[-1]})

    update_plot()
    print(f"Episode: {episode + 1}, Total Reward: {total_reward}, Loss: {losses[-1]}, Accuracy: {accuracies[-1]}")

# Save the trained model
save_path = '/home/sit-research/Desktop/DRL/combine results/revised data/DRL Data/my_dqn_model2.h5'
agent.model.save(save_path)

plt.ioff()
plt.savefig('training_metrics.png')
plt.show()


/home/sit-research/anaconda3/envs/tfenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-08-13 16:47:00.736583: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7057 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:65:00.0, compute capability: 6.1


NameError: name 'csv' is not defined